# Task

Parse all mobile models with all characteristics from https://www.gsmarena.com/

In [1]:
#pip install jupyter_contrib_nbextensions

In [2]:
#!jupyter contrib nbextension install --user

In [231]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
from random import randint
from time import sleep
import logging

In [7]:
URL_TEMPLATE = "https://www.gsmarena.com/acer-phones-59.php"
r = requests.get(URL_TEMPLATE)
print(r.status_code)
soup = bs(r.text, "html.parser")

200


# Parse brands

In [8]:
URL_TEMPLATE = "https://www.gsmarena.com/makers.php3"
r = requests.get(URL_TEMPLATE)
print(r.status_code)

200


In [9]:
soup = bs(r.text, "html.parser")

In [10]:
brands = soup.find_all('a')

In [11]:
result_list = {'href': []}

In [12]:
for name in brands:
    try:
        result_list['href'].append('https://www.gsmarena.com/'+ re.search('href="(.*)"', str(name)).group(1).split('"')[0])
    #     print(re.search('href="(.*)"', str(name)).group(1).split('"')[0])
    except AttributeError:
        continue

In [13]:
df = pd.DataFrame(result_list, columns=['href'])
df = df[df['href'].str.contains('phones')]
df = df.reset_index(drop=True)
df['brand'] ='NULL'

In [14]:
for index, value in enumerate(df['href']):
    df['brand'][index] = df['href'][index].split('/')[3].split('-')[0]

In [15]:
df.head()

,href,brand
0,https://www.gsmarena.com/acer-phones-59.php,acer
1,https://www.gsmarena.com/alcatel-phones-5.php,alcatel
2,https://www.gsmarena.com/allview-phones-88.php,allview
3,https://www.gsmarena.com/amazon-phones-76.php,amazon
4,https://www.gsmarena.com/amoi-phones-28.php,amoi


# Parse all brands, go via pagination

In [147]:
list_all_brands =  {'href': [], 'brand': []}

In [148]:
for index, value in enumerate(df['href']):
    for num in range(2,16):
        list_all_brands['href'].append(df['href'][index].split('phones')[0] +'phones-f'+ df['href'][index].split('phones')[1].split('.')[0] + '-0-p' + str(num) + '.php')
        list_all_brands['brand'].append(df['href'][index].split('/')[3].split('-')[0])


In [149]:
df_2 =  pd.DataFrame(list_all_brands, columns= ['href','brand'])
df_2

,href,brand
0,https://www.gsmarena.com/acer-phones-f-59-0-p2...,acer
1,https://www.gsmarena.com/acer-phones-f-59-0-p3...,acer
2,https://www.gsmarena.com/acer-phones-f-59-0-p4...,acer
3,https://www.gsmarena.com/acer-phones-f-59-0-p5...,acer
4,https://www.gsmarena.com/acer-phones-f-59-0-p6...,acer
...,...,...
2165,https://www.gsmarena.com/plum-phones-f-72-0-p1...,plum
2166,https://www.gsmarena.com/plum-phones-f-72-0-p1...,plum
2167,https://www.gsmarena.com/plum-phones-f-72-0-p1...,plum
2168,https://www.gsmarena.com/plum-phones-f-72-0-p1...,plum


# Parse 1st model page

In [221]:
models_list = {'model': [], 'source':[]}

In [1290]:
for index, value in enumerate(df['href']):
    r = requests.get(value)
    print(r.status_code,index,value)
    sleep(randint(1,5)) 
    
    #парсим
    soup = bs(r.text, "html.parser")
    models = soup.find_all('li')
    
    for name in models:
        if re.search(r"img src", str(name)):
            models_list['model'].append('https://www.gsmarena.com/'+ re.search('href="(.*)"', str(name)).group(1).split('"')[0])
            models_list['source'].append(value)
        else:
            continue    
            

In [ ]:
models_lis_df =  pd.DataFrame(models_list, columns=['model','source'])
models_lis_df['brand'] = 'NULL'
for index, value in enumerate(models_lis_df['source']):
    models_lis_df['brand'][index] = models_lis_df['source'][index].split('/')[3].split('-')[0]

In [619]:
models_lis_df.to_csv('parsed_devices_links_13feb23.csv')

# Parse all model pages 

In [1150]:
models_list_pag = {'model': [], 'source':[], 'brand':[]}

In [1]:
for index_b, value_b in enumerate(df_2['brand'].drop_duplicates()):
    
    for index, value in enumerate(df_2['href'][df_2['brand'] == value_b]):
        r = requests.get(value)
        print(r.status_code,index,value)
        sleep(randint(1,5)) 
        soup = bs(r.text, "html.parser")
        models = soup.find_all('li')
        
        if type(re.search(r"img src", str(models))) is type(None):
            break 
        else:
            for name in models:
                if type(re.search(r"img src", str(name))) is not type(None):
                    models_list_pag['model'].append('https://www.gsmarena.com/'+ re.search('href="(.*)"', str(name)).group(1).split('"')[0])
                    models_list_pag['source'].append(value)
                    models_list_pag['brand'].append(value_b)
                else:
                    continue    
            

In [1152]:
models_lis_df_others =  pd.DataFrame(models_list_pag, columns= ['model','source'])

In [1153]:
models_lis_df_others = models_lis_df_others.drop_duplicates()  

In [1154]:
models_lis_df_others['brand'] = 'NULL'
for index, value in enumerate(models_lis_df_others['source']):
    models_lis_df_others['brand'][index] = models_lis_df_others['source'][index].split('/')[3].split('-')[0]

In [161]:
models_lis_df_others.to_csv('models_lis_df_others.csv')

# Open every model page and get info from such website: from 1st page

Due to the fact that there are a lot of requests and they began to block me by IP, I connected this service https://app.zyte.com/. You need to link the card, but there is a 14-day free trial.

In [1289]:
models_lis_df = pd.read_csv("models_lis_df_others.csv")
models_lis_df = models_lis_df.drop(columns=['Unnamed: 0'])
models_lis_df = models_lis_df.drop_duplicates()  
models_lis_df

In [1147]:
data_model_raw = pd.DataFrame({
    'data_spec': [None] ,
    'data_spec_value': [None]  ,
    'model': [None],
    'model_name':[None]
})

In [1148]:
# html_data_model_raw = pd.DataFrame({
#     'model': [None],
#     'html':[None]
# })

In [1149]:
# сортировка
models_lis_df = models_lis_df.sort_values(by='model', ascending=True, inplace=False, na_position='last')
models_lis_df = models_lis_df.reset_index(drop=True)

In [1288]:
for global_index, model_value in enumerate(models_lis_df['model']):
    #подключаемся
    r = requests.get(model_value,
                     proxies={
        "http": "",
        "https": "",
    },
    verify='' )
    soup = bs(r.text, "html.parser")
    print(r.status_code,global_index,model_value)
  #  sleep(randint(0,1))
    
    #парсим, выкидывая строчки, где падают ошибки - обычно это строчки в которых все поля незаполнены + сохраняем всю страницу на всякий
    data_model = soup.find_all('td',"nfo",'data-spec')
    model_name = soup.find_all('title')
    df3= pd.DataFrame( {
                     "model": [model_value],
                     "html": str(soup)
                    })
    html_data_model_raw = html_data_model_raw.append(df3, ignore_index = True)
    
    for index,value in enumerate(data_model):
        try:
            df2 = pd.DataFrame( {
                     "data_spec": [re.search('data-spec="(.*)"', str(value)).group(1).split('"')[0]],
                     "data_spec_value":re.search('<td.*>(.*)</td>', str(value)).group(1).split('"')[0], 
                     "model": model_value,
                     "model_name": re.search('<title.*>(.*)</title>', str(model_name)).group(1).split(' -')[0] })
            data_model_raw = data_model_raw.append(df2, ignore_index = True)
        except AttributeError:
            continue

In [1287]:
# # для парсинга из таблички
# for global_index, model_value in enumerate(html_data_model_raw['model']):
#     #подключаемся
    
#     soup = bs(html_data_model_raw['html'][global_index], "html.parser")
#     print(global_index,model_value)
    
#     #парсим, выкидывая строчки, где падают ошибки - обычно это строчки в которых все поля незаполнены + сохраняем всю страницу на всякий
#     data_model = soup.find_all('td',"nfo",'data-spec')
#     model_name = soup.find_all('title')
#     df3= pd.DataFrame( {
#                      "model": [model_value],
#                      "html": str(soup)
#                     })
#     html_data_model_raw = html_data_model_raw.append(df3, ignore_index = True)
    
#     for index,value in enumerate(data_model):
#         try:
#             df2 = pd.DataFrame( {
#                      "data_spec": [re.search('data-spec="(.*)"', str(value)).group(1).split('"')[0]],
#                      "data_spec_value":re.search('<td.*>(.*)</td>', str(value)).group(1).split('"')[0], 
#                      "model": model_value,
#                      "model_name": re.search('<title.*>(.*)</title>', str(model_name)).group(1).split(' -')[0] })
#             data_model_raw = data_model_raw.append(df2, ignore_index = True)
#         except AttributeError:
#             continue

In [365]:
# models_lis_df[(models_lis_df.model == 'https://www.gsmarena.com/oppo_a58x-12027.php')]

,model,source,brand
1924,https://www.gsmarena.com/oppo_a58x-12027.php,https://www.gsmarena.com/oppo-phones-82.php,oppo


In [417]:
data_model_raw = data_model_raw.drop_duplicates()
data_model_raw

,data_spec,data_spec_value,model,model_name
0,None,None,None,None
1,nettech,,https://www.gsmarena.com/zte_voyage_40_pro+-12...,ZTE Voyage 40 Pro+
2,net2g,GSM 850 / 900 / 1800 / 1900 - SIM 1 &amp; SIM 2,https://www.gsmarena.com/zte_voyage_40_pro+-12...,ZTE Voyage 40 Pro+
3,net3g,HSDPA 850 / 900 / 1900 / 2100,https://www.gsmarena.com/zte_voyage_40_pro+-12...,ZTE Voyage 40 Pro+
4,net4g,LTE,https://www.gsmarena.com/zte_voyage_40_pro+-12...,ZTE Voyage 40 Pro+
...,...,...,...,...
109941,sensors,"Fingerprint (side-mounted), accelerometer, gyr...",https://www.gsmarena.com/zte_voyage_20_pro-112...,ZTE Voyage 20 Pro
109942,batdescription1,"Li-Po 5100 mAh, non-removable",https://www.gsmarena.com/zte_voyage_20_pro-112...,ZTE Voyage 20 Pro
109943,colors,"Black, Aurora",https://www.gsmarena.com/zte_voyage_20_pro-112...,ZTE Voyage 20 Pro
109944,models,9040N,https://www.gsmarena.com/zte_voyage_20_pro-112...,ZTE Voyage 20 Pro


In [434]:
data_model_raw = data_model_raw.merge(models_lis_df, left_on='model', right_on='model')

In [440]:
device_features = pd.pivot_table(data_model_raw,
               values='data_spec_value',
               index=['model', 'model_name','brand','source'],
               columns=['data_spec']
               ,aggfunc='first'
              ).reset_index()

In [446]:
device_features = device_features.rename(columns={'model': 'model_url'})
device_features = device_features.rename(columns={'data_spec': 'index'})

In [471]:
device_features['price_euro'] = 'NULL'
for x in range(len(device_features)):
    try:
        device_features['price_euro'][x] = device_features['price'][x].split(' ')[1]
    except AttributeError:
        continue
    except IndexError:
        continue

In [478]:
device_features['release_year'] = 'NULL'
for x in range(len(device_features)):
    try:
        device_features['release_year'][x] = device_features['year'][x].split(',')[0]
    except AttributeError:
        continue
    except IndexError:
        continue

In [452]:
html_data_model_raw.to_csv('html_data_model_raw.csv')

In [454]:
data_model_raw.to_csv('parsed_model_raw.csv')

In [480]:
device_features.to_csv('device_features.csv')

# Open every model page and get info from such website: from all other pages

In [1187]:
models_lis_df_pag = pd.read_csv("models_lis_df_others.csv")
models_lis_df_pag = models_lis_df_pag.drop(columns=['Unnamed: 0'])
models_lis_df_pag = models_lis_df_pag.drop_duplicates().reset_index(drop=True)  
models_lis_df_pag

,model,source
0,https://www.gsmarena.com/acer_liquid_e3-6116.php,https://www.gsmarena.com/acer-phones-f-59-0-p2...
1,https://www.gsmarena.com/acer_liquid_z4-6115.php,https://www.gsmarena.com/acer-phones-f-59-0-p2...
2,https://www.gsmarena.com/acer_iconia_b1_721-59...,https://www.gsmarena.com/acer-phones-f-59-0-p2...
3,https://www.gsmarena.com/acer_iconia_b1_720-59...,https://www.gsmarena.com/acer-phones-f-59-0-p2...
4,https://www.gsmarena.com/acer_iconia_a1_830-59...,https://www.gsmarena.com/acer-phones-f-59-0-p2...
...,...,...
8730,https://www.gsmarena.com/ulefone_s1-10036.php,https://www.gsmarena.com/ulefone_-phones-f-124...
8731,https://www.gsmarena.com/ulefone_armor_x-10037...,https://www.gsmarena.com/ulefone_-phones-f-124...
8732,https://www.gsmarena.com/ulefone_armor_2-10038...,https://www.gsmarena.com/ulefone_-phones-f-124...
8733,https://www.gsmarena.com/ulefone_armor_8_5g-10...,https://www.gsmarena.com/ulefone_-phones-f-124...


In [1221]:
data_model_raw_pag = pd.DataFrame({
    'data_spec': [None] ,
    'data_spec_value': [None]  ,
    'model': [None],
    'model_name':[None]
})

In [1222]:
html_data_model_raw_pag = pd.DataFrame({
    'model': [None],
    'html':[None]
})

In [1223]:
# сортировка
models_lis_df_pag = models_lis_df_pag.sort_values(by='model', ascending=True, inplace=False, na_position='last')
models_lis_df_pag = models_lis_df_pag.reset_index(drop=True)

In [1224]:
# складываем ссылки, которые долго грузились сюда
models_list_pag_errors = {'model': []}

In [1291]:
for global_index, model_value in enumerate(models_lis_df_pag['model']): #lost_models['model']   # models_list_pag_errors['model']
    try: 
        #подключаемся
        r = requests.get(model_value,
                         proxies={
               "",
               "",
        },
        verify='' , timeout=30.0)
        soup = bs(r.text, "html.parser")
        print(r.status_code,global_index,model_value)
      #  sleep(randint(0,1))
    
        if r.status_code != 200:
            break
        else:
            #парсим, выкидывая строчки, где падают ошибки - обычно это строчки в которых все поля незаполнены + сохраняем всю страницу на всякий
            data_model = soup.find_all('td',"nfo",'data-spec')
            model_name = soup.find_all('title')
            df3= pd.DataFrame( {
                             "model": [model_value],
                             "html": str(soup)
                            })
            html_data_model_raw_pag = html_data_model_raw_pag.append(df3, ignore_index = True)

            for index,value in enumerate(data_model):
                try:
                    df2 = pd.DataFrame( {
                             "data_spec": [re.search('data-spec="(.*)"', str(value)).group(1).split('"')[0]],
                             "data_spec_value":re.search('<td.*>(.*)</td>', str(value)).group(1).split('"')[0], 
                             "model": model_value,
                             "model_name": re.search('<title.*>(.*)</title>', str(model_name)).group(1).split(' -')[0] })
                    data_model_raw_pag = data_model_raw_pag.append(df2, ignore_index = True)
                except AttributeError:
                    continue
    except requests.Timeout as err:
        print(f'🔴 timeout error {model_value} {global_index}. Continuing loop...🔴')
        models_list_pag_errors['model'].append(model_value)
        continue

In [1226]:
models_list_pag_errors['model']

['https://www.gsmarena.com/nokia_6110_navigator-1861.php']

In [315]:
models_lis_df[(models_lis_df.model == data_model_raw_pag['model'].iloc[-1])]

,model,source
9681,https://www.gsmarena.com/zte_zmax_pro-8096.php,https://www.gsmarena.com/zte-phones-f-62-0-p4.php


In [1227]:
data_model_raw_pag

,data_spec,data_spec_value,model,model_name
0,None,None,None,None
1,nettech,,https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282
2,net2g,GSM 850 / 1900,https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282
3,gprstext,No,https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282
4,edge,No,https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282
...,...,...,...,...
30134,batstandby1,Up to 300 h,https://www.gsmarena.com/sony_ericsson_k500-84...,Sony Ericsson K500
30135,battalktime1,Up to 7 h,https://www.gsmarena.com/sony_ericsson_k500-84...,Sony Ericsson K500
30136,colors,"2 - Virtual Silver, Cosmetic Blue",https://www.gsmarena.com/sony_ericsson_k500-84...,Sony Ericsson K500
30137,sar-us,0.65 W/kg (head) 1.39 W/kg (body),https://www.gsmarena.com/sony_ericsson_k500-84...,Sony Ericsson K500


In [1228]:
data_model_raw_pag = data_model_raw_pag.drop_duplicates(subset=['model_name', 'model','data_spec_value','data_spec'])
data_model_raw_pag

,data_spec,data_spec_value,model,model_name
0,None,None,None,None
1,nettech,,https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282
2,net2g,GSM 850 / 1900,https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282
3,gprstext,No,https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282
4,edge,No,https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282
...,...,...,...,...
30134,batstandby1,Up to 300 h,https://www.gsmarena.com/sony_ericsson_k500-84...,Sony Ericsson K500
30135,battalktime1,Up to 7 h,https://www.gsmarena.com/sony_ericsson_k500-84...,Sony Ericsson K500
30136,colors,"2 - Virtual Silver, Cosmetic Blue",https://www.gsmarena.com/sony_ericsson_k500-84...,Sony Ericsson K500
30137,sar-us,0.65 W/kg (head) 1.39 W/kg (body),https://www.gsmarena.com/sony_ericsson_k500-84...,Sony Ericsson K500


In [1230]:
data_model_raw_pag = data_model_raw_pag.merge(models_lis_df_pag, left_on='model', right_on='model')
data_model_raw_pag

,data_spec,data_spec_value,model,model_name,source_x,test_x,source_y,test_y
0,nettech,,https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282,https://www.gsmarena.com/alcatel-phones-f-5-0-...,0,https://www.gsmarena.com/alcatel-phones-f-5-0-...,0
1,net2g,GSM 850 / 1900,https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282,https://www.gsmarena.com/alcatel-phones-f-5-0-...,0,https://www.gsmarena.com/alcatel-phones-f-5-0-...,0
2,gprstext,No,https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282,https://www.gsmarena.com/alcatel-phones-f-5-0-...,0,https://www.gsmarena.com/alcatel-phones-f-5-0-...,0
3,edge,No,https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282,https://www.gsmarena.com/alcatel-phones-f-5-0-...,0,https://www.gsmarena.com/alcatel-phones-f-5-0-...,0
4,year,"2011, December. Released 2011, December",https://www.gsmarena.com/alcatel_ot_282-4722.php,alcatel OT-282,https://www.gsmarena.com/alcatel-phones-f-5-0-...,0,https://www.gsmarena.com/alcatel-phones-f-5-0-...,0
...,...,...,...,...,...,...,...,...
30133,batstandby1,Up to 300 h,https://www.gsmarena.com/sony_ericsson_k500-84...,Sony Ericsson K500,https://www.gsmarena.com/sony_ericsson-phones-...,0,https://www.gsmarena.com/sony_ericsson-phones-...,0
30134,battalktime1,Up to 7 h,https://www.gsmarena.com/sony_ericsson_k500-84...,Sony Ericsson K500,https://www.gsmarena.com/sony_ericsson-phones-...,0,https://www.gsmarena.com/sony_ericsson-phones-...,0
30135,colors,"2 - Virtual Silver, Cosmetic Blue",https://www.gsmarena.com/sony_ericsson_k500-84...,Sony Ericsson K500,https://www.gsmarena.com/sony_ericsson-phones-...,0,https://www.gsmarena.com/sony_ericsson-phones-...,0
30136,sar-us,0.65 W/kg (head) 1.39 W/kg (body),https://www.gsmarena.com/sony_ericsson_k500-84...,Sony Ericsson K500,https://www.gsmarena.com/sony_ericsson-phones-...,0,https://www.gsmarena.com/sony_ericsson-phones-...,0


In [1231]:
device_features = pd.pivot_table(data_model_raw_pag,
               values='data_spec_value',
               index=['model', 'model_name'],
               columns=['data_spec']
               ,aggfunc='first'
              ).reset_index()

In [1232]:
device_features = device_features.rename(columns={'model': 'model_url'})
device_features = device_features.rename(columns={'data_spec': 'index'})

In [1233]:
device_features['price_euro'] = 'NULL'
for x in range(len(device_features)):
    try:
        device_features['price_euro'][x] = device_features['price'][x].split(' ')[1]
    except AttributeError:
        continue
    except IndexError:
        continue

In [1234]:
device_features['release_year'] = 'NULL'
for x in range(len(device_features)):
    try:
        device_features['release_year'][x] = device_features['year'][x].split(',')[0]
    except AttributeError:
        continue
    except IndexError:
        continue

In [1241]:
html_data_model_raw_pag

,model,html
0,None,None
1,https://www.gsmarena.com/alcatel_ot_282-4722.php,"<!DOCTYPE html>\n\n<html lang=""en-US"" xml:lang..."
2,https://www.gsmarena.com/alcatel_ot_292-4721.php,"<!DOCTYPE html>\n\n<html lang=""en-US"" xml:lang..."
3,https://www.gsmarena.com/alcatel_ot_228-4720.php,"<!DOCTYPE html>\n\n<html lang=""en-US"" xml:lang..."
4,https://www.gsmarena.com/alcatel_ot_223-3813.php,"<!DOCTYPE html>\n\n<html lang=""en-US"" xml:lang..."
...,...,...
952,https://www.gsmarena.com/sony_ericsson_k610-14...,"<!DOCTYPE html>\n\n<html lang=""en-US"" xml:lang..."
953,https://www.gsmarena.com/sony_ericsson_k608-12...,"<!DOCTYPE html>\n\n<html lang=""en-US"" xml:lang..."
954,https://www.gsmarena.com/sony_ericsson_k600-10...,"<!DOCTYPE html>\n\n<html lang=""en-US"" xml:lang..."
955,https://www.gsmarena.com/sony_ericsson_k508-11...,"<!DOCTYPE html>\n\n<html lang=""en-US"" xml:lang..."


In [1239]:
device_features.to_csv('device_features_pag_lost.csv')

In [1243]:
html_data_model_raw_pag.to_csv('html_data_model_raw_pag_lost.csv')

In [1240]:
device_features_pag_lost = device_features

In [334]:
# html_data_model_raw_pag.to_csv('html_data_model_raw_pag.csv')

In [335]:
# data_model_raw_pag.to_csv('parsed_model_raw_pag.csv')

In [337]:
# device_features.to_csv('device_features_pag.csv')

In [1281]:
# html_data_model_raw_all = pd.read_csv('html_data_model_raw_all.csv')

In [1282]:
# html_data_model_raw_all  = html_data_model_raw.append(html_data_model_raw_pag_lost)

/var/folders/vt/7dt72kxx6033ddhgydwhpq5w0000gn/T/ipykernel_90896/1674389282.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  html_data_model_raw_all  = html_data_model_raw.append(html_data_model_raw_pag_lost)


In [1283]:
# html_data_model_raw_all = pd.read_csv("html_data_model_raw_all.csv")

# Join 1st page and others

In [338]:
# device_features = pd.read_csv("device_features.csv")

In [1137]:
# data_model_raw = pd.read_csv("parsed_model_raw.csv")

In [341]:
# device_features_pag = pd.read_csv("device_features_pag.csv")

In [1280]:
# html_data_model_raw_pag = pd.read_csv('html_data_model_raw_pag.csv')

In [356]:
# html_data_model_raw = pd.read_csv('html_data_model_raw.csv')

In [342]:
# device_features_all = device_features.append(device_features_pag)

/var/folders/vt/7dt72kxx6033ddhgydwhpq5w0000gn/T/ipykernel_90896/1232211332.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  device_features_all = device_features.append(device_features_pag)


In [343]:
# data_model_raw_all  = data_model_raw.append(data_model_raw_pag)

/var/folders/vt/7dt72kxx6033ddhgydwhpq5w0000gn/T/ipykernel_90896/656942462.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_model_raw_all  = data_model_raw.append(data_model_raw_pag)


In [357]:
# html_data_model_raw_all  = html_data_model_raw.append(html_data_model_raw_pag)

/var/folders/vt/7dt72kxx6033ddhgydwhpq5w0000gn/T/ipykernel_90896/546757857.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  html_data_model_raw_all  = html_data_model_raw.append(html_data_model_raw_pag)


In [346]:
device_features_all.to_csv('device_features_all.csv')

In [347]:
data_model_raw_all.to_csv('data_model_raw_all.csv')

In [358]:
html_data_model_raw_all.to_csv('html_data_model_raw_all.csv')

In [1008]:
html_data_model_raw_all = pd.read_csv('html_data_model_raw_all.csv')
html_data_model_raw_all = html_data_model_raw_all.drop_duplicates()

In [1184]:
data_model_raw_all = pd.read_csv('data_model_raw_all.csv')
data_model_raw_all = data_model_raw_all.drop_duplicates(subset=['model_name', 'model_url'])
data_model_raw_all

/var/folders/vt/7dt72kxx6033ddhgydwhpq5w0000gn/T/ipykernel_90896/3372540482.py:1: DtypeWarning: Columns (6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data_model_raw_all = pd.read_csv('/Users/mikhailmerkurev/Downloads/data_model_raw_all.csv')


,Unnamed: 0.1,Unnamed: 0,data_spec,data_spec_value,model,model_name,source,brand,source_x,source_y
0,0,0.0,nettech,NaN,https://www.gsmarena.com/zte_voyage_40_pro+-12...,ZTE Voyage 40 Pro+,https://www.gsmarena.com/zte-phones-62.php,zte,NaN,NaN
1,1,1.0,net2g,GSM 850 / 900 / 1800 / 1900 - SIM 1 &amp; SIM 2,https://www.gsmarena.com/zte_voyage_40_pro+-12...,ZTE Voyage 40 Pro+,https://www.gsmarena.com/zte-phones-62.php,zte,NaN,NaN
2,2,2.0,net3g,HSDPA 850 / 900 / 1900 / 2100,https://www.gsmarena.com/zte_voyage_40_pro+-12...,ZTE Voyage 40 Pro+,https://www.gsmarena.com/zte-phones-62.php,zte,NaN,NaN
3,3,3.0,net4g,LTE,https://www.gsmarena.com/zte_voyage_40_pro+-12...,ZTE Voyage 40 Pro+,https://www.gsmarena.com/zte-phones-62.php,zte,NaN,NaN
4,4,4.0,net5g,"1, 5, 28, 41, 78, 84 SA/NSA",https://www.gsmarena.com/zte_voyage_40_pro+-12...,ZTE Voyage 40 Pro+,https://www.gsmarena.com/zte-phones-62.php,zte,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
370149,261513,NaN,batmusicplayback1,Up to 32 h,https://www.gsmarena.com/vivo_xshot-6799.php,vivo Xshot,NaN,NaN,NaN,https://www.gsmarena.com/vivo-phones-f-98-0-p9...
370150,261514,NaN,colors,White/Black,https://www.gsmarena.com/vivo_xshot-6799.php,vivo Xshot,NaN,NaN,NaN,https://www.gsmarena.com/vivo-phones-f-98-0-p9...
370151,261515,NaN,models,X710L,https://www.gsmarena.com/vivo_xshot-6799.php,vivo Xshot,NaN,NaN,NaN,https://www.gsmarena.com/vivo-phones-f-98-0-p9...
370152,261516,NaN,sar-us,0.47 W/kg (head) 1.33 W/kg (body),https://www.gsmarena.com/vivo_xshot-6799.php,vivo Xshot,NaN,NaN,NaN,https://www.gsmarena.com/vivo-phones-f-98-0-p9...


# Enrich dataset with prices

In [1245]:
price_data = pd.DataFrame({
    'model': [None] ,
    'price': [None] 
   
})

In [1247]:
# html_data_model_raw_pag_lost = html_data_model_raw_pag_lost.drop_duplicates()

In [1248]:
# html_data_model_raw_pag_lost = html_data_model_raw_pag_lost.reset_index(drop=True)

In [1011]:
html_data_model_raw_all = html_data_model_raw_all.drop_duplicates()

In [1012]:
html_data_model_raw_all = html_data_model_raw_all.reset_index(drop=True)

In [1292]:
# для парсинга из таблички
for global_index, model_value in enumerate(html_data_model_raw_pag_lost['model']):  # html_data_model_raw_all['model']
 
    #подключаемся
    try:
        soup = bs(html_data_model_raw_pag_lost['html'][global_index], "html.parser")  # html_data_model_raw_all['html']
        print(global_index,model_value)

        #парсим, выкидывая строчки, где падают ошибки - обычно это строчки в которых все поля незаполнены + сохраняем всю страницу на всякий
        data_model = soup.find_all('td',"nfo",'data-spec')        

        #цену в отдельный датафрейм загоняем 
        try:
            price_raw = re.search('data-spec="price(.*)', str(data_model)).group(1)

            if 'EUR' in price_raw:
                df2 = pd.DataFrame( {
                    "model": [model_value],
                     "price": re.search('>(.*)<', price_raw).group(1).split()[1], 
                     })
                price_data = price_data.append(df2, ignore_index = True)

            else:
                x = re.search('<a(.*)/a>', price_raw).group(1)
                array_price = re.search('>(.*)<',x).group(1).replace('\u2009','').replace(',','').split(r' / ')
                df2 = pd.DataFrame( {
                    "model": [model_value],
                     "price": [array_price], 
                     })
                price_data = price_data.append(df2, ignore_index = True)



        except AttributeError:
            df2 = pd.DataFrame( {
                    "model": [model_value],
                    "price": re.search('>(.*)<', price_raw).group(1).split()[1], })
            price_data = price_data.append(df2, ignore_index = True)
            continue
            
    except TypeError:
        df2 = pd.DataFrame( {
                    "model": [model_value],
                     "price": 0, 
                     })
        price_data = price_data.append(df2, ignore_index = True)
        continue


In [1251]:
price_data_df = price_data

In [1252]:
price_data_df['euro_total'] = 0
price_data_df['$'] = 0
price_data_df['€'] = 0
price_data_df['£'] = 0

for index, x in enumerate(price_data_df['price']):
    try:
        for element in x:
                if '$' in element:
                    price_data_df['$'][index] = element[1:].split('.')[0]
                if '€' in element:
                    price_data_df['€'][index] = element[1:].split('.')[0]
                if '£' in element:
                    price_data_df['£'][index] = element[1:].split('.')[0]
    except TypeError:
        continue

/var/folders/vt/7dt72kxx6033ddhgydwhpq5w0000gn/T/ipykernel_90896/278357810.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data_df['$'][index] = element[1:].split('.')[0]
/var/folders/vt/7dt72kxx6033ddhgydwhpq5w0000gn/T/ipykernel_90896/278357810.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data_df['£'][index] = element[1:].split('.')[0]
/var/folders/vt/7dt72kxx6033ddhgydwhpq5w0000gn/T/ipykernel_90896/278357810.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [1254]:
for index, x in enumerate(price_data_df['price']):
    if price_data_df['$'][index] == 0 and price_data_df['€'][index] ==0 and price_data_df['£'][index] ==0 :
        price_data_df['euro_total'][index] = price_data_df['price'][index]

In [1255]:
for index, x in enumerate(price_data_df['price']):
    if price_data_df['€'][index] !=0:
        price_data_df['euro_total'][index] = price_data_df['€'][index]

In [1256]:
for index, x in enumerate(price_data_df['price']):
    try:
        if price_data_df['euro_total'][index] == 0 :
            price_data_df['euro_total'][index] = int(price_data_df['$'][index])* 0.94 # по курсу на 17.02.23
    except ValueError: 
        try:
            price_data_df['euro_total'][index] = int(price_data_df['$'][index][1:])* 0.94 # по курсу на 17.02.23
            continue
        except ValueError: 
            price_data_df['euro_total'][index] = 0

In [1257]:
for index, x in enumerate(price_data_df['price']):
    try:
        if price_data_df['euro_total'][index] == 0 :
            price_data_df['euro_total'][index] = int(price_data_df['£'][index])* 1.12 # по курсу на 17.02.23
    except ValueError: 
        try:
            price_data_df['euro_total'][index] = int(price_data_df['£'][index][1:])* 1.12 # по курсу на 17.02.23
            continue
        except ValueError: 
            price_data_df['euro_total'][index] = 0

In [1258]:
for index, x in enumerate(price_data_df['euro_total']):
    try:
        price_data_df['euro_total'][index] = float(price_data_df['euro_total'][index])
    except ValueError: 
        price_data_df['euro_total'][index] = 0
    except TypeError: 
        price_data_df['euro_total'][index] = 0

In [1032]:
# price_data_df.to_csv('price_data_df.csv')

In [1127]:
device_features_all = pd.read_csv('device_features_all.csv')

In [1128]:
device_features_all = device_features_all.merge(price_data_df, left_on='model_url', right_on='model')
device_features_all = device_features_all.drop(columns=['Unnamed: 0.1','price_euro','price_y','Unnamed: 0','source'])

In [1129]:
device_features_all = device_features_all.rename(columns={'price_x': 'price'})

In [1130]:
# добавляем удобное имя для парсинга
device_features_all['model_name_new'] = 'NULL'
device_features_all['model_name_new_lower'] = 'NULL'
for x in range(len(device_features_all['model_name'])):
    try:
        limited_name = device_features_all['model_name'][x].split(" ")[1:]
        device_features_all['model_name_new'][x] = ' '.join(limited_name).replace("(", "" ).replace(")", "" )
#         device_features_all['model_name_new_lower'][x] = ' '.join(limited_name).replace("(", "" ).replace(")", "" ).lower()
    except AttributeError:
        continue
    except IndexError:
        continue

/var/folders/vt/7dt72kxx6033ddhgydwhpq5w0000gn/T/ipykernel_90896/354211543.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  device_features_all['model_name_new'][x] = ' '.join(limited_name).replace("(", "" ).replace(")", "" )


In [1131]:
device_features_all['brand'] = 'NULL'
for index, value in enumerate(device_features_all['model_url']):
    device_features_all['brand'][index] = device_features_all['model_url'][index].split('/')[3].split('_')[0].capitalize()

/var/folders/vt/7dt72kxx6033ddhgydwhpq5w0000gn/T/ipykernel_90896/3122488368.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  device_features_all['brand'][index] = device_features_all['model_url'][index].split('/')[3].split('_')[0].capitalize()


In [1132]:
device_features_all.drop_duplicates(subset=['model_name', 'model_url'], inplace=True)

In [1133]:
device_features_all.reset_index(inplace=True, drop=True)

In [1277]:
device_features_all.to_csv('device_features_all_2.csv')